## investigations
Conclusions on mods:

- Freddie data only flags mod on the month it occurs, Fannie keeps the flag for the duration of the mod.
- The mod_sticky_flg unifies the data: it is always Y after a mod
- Mods have significant prior dq

Conclusions on borr_asst_plan:

- Flags are sticky
- Can have multiple episodes of plans
- Some but not a lot of overlap with mods
- F: most were had at most months_dq=1
- R: most were 1+ months_dq prior, mostly used by Freddie
- Tend to be short term in nature

Conclusions on interest rate reductions:

- Almost all ir reductions are marked as mods
- mods involve ir reduction about 60% of the time


In [1]:
import mortgage_imports.clickhouse_utilities as cu
import pandas as pd
import numpy as np
from muti import chu

In [2]:
# pandas options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.max_rows', 1000)
client = chu.make_connection()

In [3]:
qry = \
"""
SELECT
    src_data,
    m.mod_flg AS mf,
    m.mod_sticky_flg AS msf,
    COUNT(*) AS ln_mon
FROM
    unified.frannie ARRAY JOIN monthly AS m
GROUP BY
  src_data,
  mf,
  msf
ORDER BY
  src_data,
  mf,
  msf;
"""

df1 = chu.run_query(qry, client, return_df=True)
df1.head(n=100)

,src_data,mf,msf,ln_mon
0,fannie,!,!,33987601
1,fannie,!,Y,368602
2,fannie,N,N,2386144619
3,fannie,N,Y,119336
4,fannie,Y,Y,38026197
5,freddie:non-standard,N,N,329100922
6,freddie:non-standard,N,Y,16942726
7,freddie:non-standard,Y,Y,307970
8,freddie:standard,N,N,2002062338
9,freddie:standard,N,Y,25874298


In [4]:
# look at relations between mod_flg and dq on that date and max prior dq
# most frequently they are reset to current at the mod date and were 6+ months dq prior
qry = \
"""
SELECT
  src_data,
  prior_dq > 6 ? 6 : prior_dq AS prior_dq,
  count(*) AS nl
FROM (
    SELECT
        src_data,
        arrayMax(arraySlice(monthly.months_dq, 1, indexOf(monthly.mod_flg, 'Y'))) AS prior_dq
    FROM
        unified.frannie
    WHERE
        has(monthly.mod_flg, 'Y'))
GROUP BY src_data, prior_dq
ORDER BY src_data, prior_dq
"""

df2 = chu.run_query(qry, client, return_df=True)
df2.head(n=1000)

,src_data,prior_dq,nl
0,fannie,0,754
1,fannie,1,3102
2,fannie,2,18232
3,fannie,3,45659
4,fannie,4,74404
5,fannie,5,82696
6,fannie,6,368877
7,freddie:non-standard,0,1108
8,freddie:non-standard,1,5397
9,freddie:non-standard,2,11786


you can see that the effect of the sticky mod flag--little effect on Fannie, big effect of Freddie
where the flag only appears on the month of the mod

modded loans tend to be very dq in their history

In [5]:
# look at the relationship between mod_flg and borrower assistance plan

# R = repayment plan
# F = forbearance
# T = Trial
# N = no plan
# 7/9 = Not applicable/available (fannie)

# repayment plans: Freddie does more, about 1/3 are Y (standard) for mod_sticky_flg. None for fannie.
# forbearance: vast majority are not listed as mod -- perhaps coincidental overlap (ie forbearance on
#              a loan that happened to be modified)
qry = \
"""
SELECT
    src_data,
    m.mod_flg AS mf,
    m.mod_sticky_flg AS msf,
    m.borr_asst_plan AS bap,
    count(*) AS ln_mon
FROM
    unified.frannie ARRAY JOIN monthly AS m
GROUP BY
  src_data,
  mf,
  msf,
  bap
ORDER BY
  src_data,
  mf,
  msf,
  bap;
"""

df = chu.run_query(qry, client, return_df=True)
df.sort_values(['src_data', 'bap', 'mf', 'msf'])[['src_data', 'bap', 'mf', 'msf', 'ln_mon']].head(n=1000)

,src_data,bap,mf,msf,ln_mon
0,fannie,!,!,!,30623405
2,fannie,!,!,Y,333362
4,fannie,!,N,N,2250183556
10,fannie,!,N,Y,99998
16,fannie,!,Y,Y,35892258
1,fannie,7,!,!,3364196
3,fannie,7,!,Y,35240
5,fannie,7,N,N,129464692
11,fannie,7,N,Y,14887
17,fannie,7,Y,Y,1458903


In [6]:

# look at sticky-ness of borr_asst_plan: Fannie
# sticky
# code '7' seems to indicate end of forbearance

qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'F')) AS bap
FROM
    fannie.final
WHERE
    has(monthly.borr_asst_plan, 'F')
ORDER BY rand32(1)
LIMIT 35
"""

df3 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df3['bap']))

[list(['F', 'F', 'F']) list(['F', 'F', 'F', 'F', '7', '7', '7', '7'])
 list(['F', 'F', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F']) list(['F', 'F', '7'])
 list(['F', 'F', 'F', '7', '7', '7', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', 'F', 'F', 'F', 'F', 'F']) list(['F', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', 'F', 'F', 'F', '7', 'F', '7', '7'])
 list(['F', 'F', 'F', '7', '7', '7', '7', '7', '7'])
 list(['F', 'F', '7', '7', '7', 'N', 'F', 'F'])
 list(['F', 'F', 'F', 'F', 'N', 'N', '7'])
 list(['F', 'F', '7', '7', '7', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', '7', '7', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', '7'])
 list(['F', 'F', 'F', 'F', '7', '7', '7', '7', '7'])
 list(['F', 'F', 'F', '7', '7', '7', '7', '7', '7'])
 list(['F', 'F', 'F', 'F', 'F', 

In [7]:

# look at sticky-ness of borr_asst_plan: Freddie, sticky
# becomes unpopulated after forbearance ends


qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'F')) AS bap
FROM
    freddie.final
WHERE
    has(monthly.borr_asst_plan, 'F')
ORDER BY rand32(1)
LIMIT 35
"""

df4 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df4['bap']))

[list(['F', 'F', 'F', '!', '!', '!', '!', '!'])
 list(['F', 'F', 'F', '!', '!', '!', '!', '!', '!'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', '!', '!', '!', 'F', 'F', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['F', '!', '!', '!'])
 list(['F', 'F', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', 'F', '!', 'F', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['F', 'F', 'F', 'F', 'F', 'F', 'F', '!'])
 list(['F', 'F', 'F', '!', '!', '!'])
 list(['F', 'F', 'F', 'F', '!', 'F', 'F', '!', '!'])
 list(['F', 'F', '!', '!', '!', '!', '!', '!', '!'])
 list(['F', 'F', 'F', 'F', 'F', '!', 'F', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', 

In [8]:

# look at sticky-ness of borr_asst_plan: Fannie
# sticky
# code '7' seems to indicate end of repayment

qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'R')) AS bap
FROM
    fannie.final
WHERE
    has(monthly.borr_asst_plan, 'R')
ORDER BY rand32(1)
LIMIT 35
"""

df5 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df5['bap']))

[list(['R', 'R', '7']) list(['R', 'R', 'R', 'R', 'R', 'R'])
 list(['R', 'R', 'R', 'R', 'R', 'R']) list(['R', 'R'])
 list(['R', 'R', 'R', '7', '7', '7', '7']) list(['R', 'R'])
 list(['R', 'R', 'F', 'F', 'F', 'F'])
 list(['R', 'R', 'N', '7', '7', '7', '7', '7', '7'])
 list(['R', 'R', 'R', 'R', 'R', 'R', '7'])
 list(['R', 'R', 'R', 'N', '7', '7', '7', '7', '7'])
 list(['R', 'N', 'R', 'R', 'R'])
 list(['R', '7', '7', 'N', 'N', 'N', '7', 'N', '7']) list(['R', 'R'])
 list(['R', '7', '7', '7']) list(['R', 'F', 'F', 'F', 'F', 'F', 'F'])
 list(['R']) list(['R']) list(['R', 'R', 'R', 'R', 'R', 'F', 'F'])
 list(['R', '7', '7']) list(['R', '7'])
 list(['R', 'R', 'R', 'N', 'R', 'N', '7', '7', 'N', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '

In [9]:

# look at sticky-ness of borr_asst_plan: Freddie
# sticky


qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'R')) AS bap
FROM
    freddie.final
WHERE
    has(monthly.borr_asst_plan, 'R')
ORDER BY rand32(1)
LIMIT 35
"""

df6 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df6['bap']))

[list(['R', '!', '!', '!', '!', '!', '!', 'T', 'T', 'T', '!', '!', 'T', '!', '!', '!', '!', '!', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', 'T', '!'])
 list(['R', 'R', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['R', 'R', '!', '!', '!', 'T', 'T', 'T', 'T', '!', '!', '!', '!', '!', 'T', 'T', 'T', 'T', 'T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['R', 'R', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!',

In [10]:

# look at sticky-ness of borr_asst_plan: Fannie
# sticky
# code '7' seems to indicate end of trial

qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'T')) AS bap
FROM
    fannie.final
WHERE
    has(monthly.borr_asst_plan, 'T')
ORDER BY rand32(1)
LIMIT 35
"""

df7 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df7['bap']))

[list(['T', 'T']) list(['T']) list(['T', '7', '7'])
 list(['T', 'T', 'T', 'T', 'N', 'N', 'N', '7']) list(['T', 'T', 'T', 'T'])
 list(['T', 'T', 'T', 'T', 'T']) list(['T', 'T', 'T', 'T', 'T', 'N'])
 list(['T', 'T', 'T']) list(['T'])
 list(['T', '7', '7', '7', '7', '7', '7', '7', '7']) list(['T', 'N'])
 list(['T', 'T', 'F', 'F', 'F', 'N']) list(['T', 'T', 'N', 'N', 'T'])
 list(['T', '7']) list(['T', 'N', 'N', 'N', '7', '7', '7', '7', '7'])
 list(['T', 'N', '7', '7', 'F', 'F', 'F', 'F', 'F']) list(['T', 'T', 'T'])
 list(['T', 'T', 'N', 'N', '7', '7', '7', '7', '7'])
 list(['T', 'F', 'F', 'F']) list(['T', 'T', 'N', 'N', '7', '7', '7', '7'])
 list(['T', 'T', 'T', 'T', '7', '7']) list(['T'])
 list(['T', 'T', 'T', 'T']) list(['T', 'T', 'T', 'N', 'N', 'N'])
 list(['T', 'T', 'T', 'N', 'N']) list(['T', 'T'])
 list(['T', 'F', 'F', 'F', '7', '7']) list(['T']) list(['T', 'T', 'T'])
 list(['T', 'N', 'N', 'T', 'T', 'T', 'T', 'N', 'N']) list(['T'])
 list(['T', 'N']) list(['T', 'N', 'N', 'T', 'N', 'T',

In [11]:

# look at sticky-ness of borr_asst_plan: Fannie
# sticky
# code '7' seems to indicate end of trial

qry = \
"""
SELECT
    arraySlice(monthly.borr_asst_plan, indexOf(monthly.borr_asst_plan, 'T')) AS bap
FROM
    freddie.final
WHERE
    has(monthly.borr_asst_plan, 'T')
ORDER BY rand32(1)
LIMIT 35
"""

df8 = chu.run_query(qry, client, return_df=True)
print(np.asarray(df8['bap']))

[list(['T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', 'T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', 'F', 'F', 'F', '!', '!', '!'])
 list(['T', 'T', 'T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!', '!'])
 list(['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', '!', '!', 'T', 'T', 'T', '!', '!', '!', '!', '!', '!', '!', '

In [19]:
# loans active 12/2020
qry = \
"""
SELECT
    has(monthly.dt, toDate('2020-12-01')) AS dec2020,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY dec2020
"""

df8b = chu.run_query(qry, client, return_df=True)
df8b.head()

,dec2020,nl
0,0,68031587
1,1,26004490


In [18]:
# loans with bap = R
qry = \
"""
SELECT
    has(monthly.borr_asst_plan, 'R') AS bap,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY bap
"""

df8c = chu.run_query(qry, client, return_df=True)
df8c.head()

,bap,nl
0,0,93877028
1,1,159049


In [17]:
# loans with bap = F
qry = \
"""
SELECT
    has(monthly.borr_asst_plan, 'F') AS bap,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY bap
"""

df8b = chu.run_query(qry, client, return_df=True)
df8b.head()

,bap,nl
0,0,92202937
1,1,1833140


In [16]:
# loans with bap = T
qry = \
"""
SELECT
    has(monthly.borr_asst_plan, 'T') AS bap,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY bap
"""

df8a = chu.run_query(qry, client, return_df=True)
df8a.head()

,bap,nl
0,0,93691931
1,1,344146


In [12]:

# look at relations between borr_asst_plan=F and max prior dq

qry = \
"""
SELECT
  src_data,
  prior_dq > 6 ? 6 : prior_dq AS prior_dq,
  count(*) AS nl
FROM (
    SELECT
        src_data,
        arrayMax(arraySlice(monthly.months_dq, 1, indexOf(monthly.borr_asst_plan, 'F'))) AS prior_dq
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'F'))
GROUP BY src_data, prior_dq
ORDER BY src_data, prior_dq
"""

df9 = chu.run_query(qry, client, return_df=True)
df9.head(n=1000)

,src_data,prior_dq,nl
0,fannie,0,389596
1,fannie,1,459035
2,fannie,2,65110
3,fannie,3,22219
4,fannie,4,13528
5,fannie,5,12753
6,fannie,6,54364
7,freddie:non-standard,0,3212
8,freddie:non-standard,1,27502
9,freddie:non-standard,2,7703


In [13]:

# look at relations between borr_asst_plan=R and max prior dq

qry = \
"""
SELECT
  src_data,
  prior_dq > 6 ? 6 : prior_dq AS prior_dq,
  count(*) AS nl
FROM (
    SELECT
        src_data,
        arrayMax(arraySlice(monthly.months_dq, 1, indexOf(monthly.borr_asst_plan, 'R'))) AS prior_dq
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'R'))
GROUP BY src_data, prior_dq
ORDER BY src_data, prior_dq
"""

df9 = chu.run_query(qry, client, return_df=True)
df9.head(n=1000)

,src_data,prior_dq,nl
0,fannie,0,639
1,fannie,1,3369
2,fannie,2,5086
3,fannie,3,3897
4,fannie,4,1062
5,fannie,5,1056
6,fannie,6,3453
7,freddie:non-standard,0,19
8,freddie:non-standard,1,2873
9,freddie:non-standard,2,4821


In [26]:
# of months bap=F distribution

qry = \
"""
SELECT
  avg(num_months)
FROM (
    SELECT
        arrayElement(monthly.borr_asst_plan, length(monthly.borr_asst_plan)) AS last_element,
        countEqual(monthly.borr_asst_plan, 'F') AS num_months
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'F')
        AND last_element != 'F')
"""

df9a = chu.run_query(qry, client, return_df=True)
df9a.head(n=1000)


,avg_num_months_
0,3.873561


In [24]:
# of months bap=F distribution

qry = \
"""
SELECT
  num_months,
  count(*) AS num_loans
FROM (
    SELECT
        arrayElement(monthly.borr_asst_plan, length(monthly.borr_asst_plan)) AS last_element,
        countEqual(monthly.borr_asst_plan, 'F') AS num_months
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'F')
        AND last_element != 'F')
GROUP BY num_months
ORDER BY num_months
"""

df9a = chu.run_query(qry, client, return_df=True)
df9a.head(n=1000)


,num_months,num_loans
0,1,138393
1,2,178454
2,3,226409
3,4,169221
4,5,132907
5,6,130958
6,7,75330
7,8,38372
8,9,4301
9,10,2185


In [25]:
# of months bap=R distribution

qry = \
"""
SELECT
  num_months,
  count(*) AS num_loans
FROM (
    SELECT
        arrayElement(monthly.borr_asst_plan, length(monthly.borr_asst_plan)) AS last_element,
        countEqual(monthly.borr_asst_plan, 'R') AS num_months
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'R')
        AND last_element != 'R')
GROUP BY num_months
ORDER BY num_months
"""

df9a = chu.run_query(qry, client, return_df=True)
df9a.head(n=1000)


,num_months,num_loans
0,1,39226
1,2,25572
2,3,19278
3,4,15451
4,5,15900
5,6,11136
6,7,5651
7,8,3812
8,9,2857
9,10,2361


In [23]:
# of months bap=R distribution

qry = \
"""
SELECT
  num_months,
  count(*) AS num_loans
FROM (
    SELECT
        arrayElement(monthly.borr_asst_plan, length(monthly.borr_asst_plan)) AS last_element,
        countEqual(monthly.borr_asst_plan, 'T') AS num_months
    FROM
        unified.frannie
    WHERE
        has(monthly.borr_asst_plan, 'T')
        AND last_element != 'T')
GROUP BY num_months
ORDER BY num_months
"""

df9a = chu.run_query(qry, client, return_df=True)
df9a.head(n=1000)



,num_months,num_loans
0,1,36340
1,2,52699
2,3,113715
3,4,46514
4,5,22784
5,6,16201
6,7,10601
7,8,7142
8,9,4580
9,10,3047


Conclusions on borr_asst_plan:

- Flags are sticky
- Can have multiple episodes of plans
- Some but not of overlap with mods
- F: most were had at most months_dq=1
- R: most were 1+ months_dq prior, mostly used by Freddie
- Tend to be short term in nature

In [14]:

# look at relations between mod_flg and interest reductions
qry = \
"""
SELECT
    src_data,
    msf,
    COUNT(*) AS ln
FROM (
    SELECT
        ln_id,
        src_data,
        ln_amort_cd,
        arrayMap((ir, zb) -> IF(zb='!', ln_orig_ir - ir, 0.0), monthly.ir, monthly.zb_cd) AS delta_ir,
        arrayMax(delta_ir) AS md,
        arrayFirstIndex(x-> IF(x > 0.25, 1, 0), delta_ir) AS fbig,
        arrayElement(monthly.mod_sticky_flg, fbig) AS msf
    FROM
        unified.frannie
    WHERE
        md > 0.25
        AND ln_amort_cd = 'FRM')
GROUP BY src_data, msf
ORDER BY src_data, msf
"""

dfa = chu.run_query(qry, client, return_df=True)
dfa.head(n=1000)

,src_data,msf,ln
0,fannie,N,6885
1,fannie,Y,367806
2,freddie:non-standard,N,13020
3,freddie:non-standard,Y,134159
4,freddie:standard,N,15014
5,freddie:standard,Y,256527


In [15]:
# look at relations between mod_flg and interest reductions
qry = \
"""
SELECT
    src_data,
    sum(IF(max_delta_ir > 0.25, 1, 0)) AS ir_reduction,
    count(*) AS ln_mods,
    sum(IF(max_delta_ir > 0.25, 1, 0)) / count(*) AS ir_reduction_rate
FROM (
    SELECT
        ln_id,
        src_data,
        ln_amort_cd,
        arrayMap((ir, zb) -> IF(zb='!', ln_orig_ir - ir, 0.0), monthly.ir, monthly.zb_cd) AS delta_ir,
        arrayMax(delta_ir) AS max_delta_ir
    FROM
        unified.frannie
    WHERE
        has(monthly.mod_sticky_flg, 'Y')
        AND ln_amort_cd = 'FRM')
GROUP BY src_data
ORDER BY src_data
"""

dfb = chu.run_query(qry, client, return_df=True)
dfb.head(n=1000)

,src_data,ir_reduction,ln_mods,ir_reduction_rate
0,fannie,368559,593724,0.620758
1,freddie:non-standard,140750,174341,0.807326
2,freddie:standard,270713,434767,0.622662


Almost all ir reductions are marked as mods
mods involve ir reduction about 60% of the time

freddie/fannie comparisons

In [45]:
# ln_purp_cd
qry = \
"""
SELECT
    src_data,
    ln_purp_cd,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY src_data, ln_purp_cd
ORDER BY src_data, ln_purp_cd
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

dfx = dfx1.groupby('src_data')['nl'].sum()
dfx.rename('tots')
dfx.name='tot'
dfx1 = dfx1.merge(dfx, on='src_data')
dfx1['distr'] = 100.0 * dfx1['nl'] / dfx1['tot']
dfx1[['src_data', 'ln_purp_cd', 'nl', 'distr']].head(n=100)

,src_data,ln_purp_cd,nl,distr
0,fannie,C,13563437,27.720440
1,fannie,P,17538824,35.845186
2,fannie,R,17812585,36.404688
3,fannie,U,14525,0.029686
4,freddie:non-standard,C,1284985,19.565635
5,freddie:non-standard,P,3454285,52.596162
6,freddie:non-standard,R,1819708,27.707516
7,freddie:non-standard,U,8583,0.130688
8,freddie:standard,C,9687698,25.137294
9,freddie:standard,P,12879922,33.420363


In [46]:
# ln_hrprog_flg
qry = \
"""
SELECT
    src_data,
    ln_hrprog_flg,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY src_data, ln_hrprog_flg
ORDER BY src_data, ln_hrprog_flg
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

dfx = dfx1.groupby('src_data')['nl'].sum()
dfx.rename('tots')
dfx.name='tot'
dfx1 = dfx1.merge(dfx, on='src_data')
dfx1['distr'] = 100.0 * dfx1['nl'] / dfx1['tot']
dfx1[['src_data', 'ln_hrprog_flg', 'nl', 'distr']].head(n=100)

,src_data,ln_hrprog_flg,nl,distr
0,fannie,!,37651429,76.950568
1,fannie,N,10688365,21.844477
2,fannie,Y,589577,1.204955
3,freddie:non-standard,H,152004,2.314467
4,freddie:non-standard,N,6415557,97.685533
5,freddie:standard,H,544867,1.413801
6,freddie:standard,N,37994278,98.586199


In [47]:

# prop_type_cd
qry = \
"""
SELECT
    src_data,
    prop_type_cd,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY src_data, prop_type_cd
ORDER BY src_data, prop_type_cd
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

dfx = dfx1.groupby('src_data')['nl'].sum()
dfx.rename('tots')
dfx.name='tot'
dfx1 = dfx1.merge(dfx, on='src_data')
dfx1['distr'] = 100.0 * dfx1['nl'] / dfx1['tot']
dfx1[['src_data', 'prop_type_cd', 'nl', 'distr']].head(n=100)

,src_data,prop_type_cd,nl,distr
0,fannie,CO,3989227,8.153031
1,fannie,CP,216089,0.441635
2,fannie,MH,337740,0.690260
3,fannie,PU,9196313,18.795077
4,fannie,SF,35190002,71.919997
5,freddie:non-standard,!,69,0.001051
6,freddie:non-standard,CO,754873,11.493963
7,freddie:non-standard,CP,13501,0.205571
8,freddie:non-standard,MH,31420,0.478412
9,freddie:non-standard,PU,988537,15.051813


In [48]:

# prop_occ_cd
qry = \
"""
SELECT
    src_data,
    prop_occ_cd,
    count(*) AS nl
FROM
    unified.frannie
GROUP BY src_data, prop_occ_cd
ORDER BY src_data, prop_occ_cd
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

dfx = dfx1.groupby('src_data')['nl'].sum()
dfx.rename('tots')
dfx.name='tot'
dfx1 = dfx1.merge(dfx, on='src_data')
dfx1['distr'] = 100.0 * dfx1['nl'] / dfx1['tot']
dfx1[['src_data', 'prop_occ_cd', 'nl', 'distr']].head(n=100)

,src_data,prop_occ_cd,nl,distr
0,fannie,I,3330499,6.806748
1,fannie,P,43671118,89.253381
2,fannie,S,1927600,3.939556
3,fannie,U,154,0.000315
4,freddie:non-standard,I,329580,5.018301
5,freddie:non-standard,P,5903110,89.882835
6,freddie:non-standard,S,326316,4.968602
7,freddie:non-standard,U,8555,0.130261
8,freddie:standard,I,2129497,5.525543
9,freddie:standard,P,34957733,90.707080


In [49]:


# ln_orig_ltv
qry = \
"""
SELECT
    src_data,
    min(ln_orig_ltv),
    avg(ln_orig_ltv),
    max(ln_orig_ltv)
FROM
    unified.frannie
WHERE
    ln_orig_ltv > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_ln_orig_ltv_,avg_ln_orig_ltv_,max_ln_orig_ltv_
0,fannie,1,71.211866,999
1,freddie:non-standard,5,76.806368,145
2,freddie:standard,1,70.777542,809


In [50]:


# ln_orig_cltv
qry = \
"""
SELECT
    src_data,
    min(ln_orig_cltv),
    avg(ln_orig_cltv),
    max(ln_orig_cltv)
FROM
    unified.frannie
WHERE
    ln_orig_cltv > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_ln_orig_cltv_,avg_ln_orig_cltv_,max_ln_orig_cltv_
0,fannie,1,72.034021,999
1,freddie:non-standard,5,78.502685,401
2,freddie:standard,1,72.000797,899


In [51]:


# borr_orig_fico
qry = \
"""
SELECT
    src_data,
    min(borr_orig_fico),
    avg(borr_orig_fico),
    max(borr_orig_fico)
FROM
    unified.frannie
WHERE
    borr_orig_fico > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_borr_orig_fico_,avg_borr_orig_fico_,max_borr_orig_fico_
0,fannie,300,745.126012,850
1,freddie:non-standard,300,720.790632,850
2,freddie:standard,300,741.210257,850


In [52]:
# borr_dti
qry = \
"""
SELECT
    src_data,
    min(borr_dti),
    avg(borr_dti),
    max(borr_dti)
FROM
    unified.frannie
WHERE
    borr_dti > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_borr_dti_,avg_borr_dti_,max_borr_dti_
0,fannie,1,33.504046,64
1,freddie:non-standard,1,34.439928,65
2,freddie:standard,1,33.015878,65


In [53]:
# ln_orig_prin
qry = \
"""
SELECT
    src_data,
    min(ln_orig_prin),
    avg(ln_orig_prin),
    max(ln_orig_prin)
FROM
    unified.frannie
WHERE
    ln_orig_prin > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_ln_orig_prin_,avg_ln_orig_prin_,max_ln_orig_prin_
0,fannie,1000.0,202958.224519,1473000.0
1,freddie:non-standard,1000.0,187646.641729,1387000.0
2,freddie:standard,1000.0,192674.690059,1473000.0


In [54]:
# ln_orig_ir
qry = \
"""
SELECT
    src_data,
    min(ln_orig_ir),
    avg(ln_orig_ir),
    max(ln_orig_ir)
FROM
    unified.frannie
WHERE
    ln_orig_ir > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_ln_orig_ir_,avg_ln_orig_ir_,max_ln_orig_ir_
0,fannie,1.75,4.965874,16.50
1,freddie:non-standard,0.50,6.015233,14.05
2,freddie:standard,1.75,5.091005,13.95


In [55]:
# ln_mi_pct
qry = \
"""
SELECT
    src_data,
    min(ln_mi_pct),
    avg(ln_mi_pct),
    max(ln_mi_pct)
FROM
    unified.frannie
WHERE
    ln_mi_pct > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,min_ln_mi_pct_,avg_ln_mi_pct_,max_ln_mi_pct_
0,fannie,0.12,24.026512,65.0
1,freddie:non-standard,1.00,25.300741,53.0
2,freddie:standard,1.00,24.069388,54.0


In [57]:

# ln_defrl_amt
qry = \
"""
SELECT
    src_data,
    count(*) as nl,
    min(ln_defrl_amt),
    avg(ln_defrl_amt),
    max(ln_defrl_amt)
FROM
    unified.frannie
WHERE
    ln_defrl_amt > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,nl,min_ln_defrl_amt_,avg_ln_defrl_amt_,max_ln_defrl_amt_
0,fannie,210800,119.580002,6707.312578,70043.328125
1,freddie:non-standard,88241,0.010000,48999.011408,527100.000000
2,freddie:standard,268046,6.000000,17445.250117,501884.343750


In [59]:


# ln_defrl_amt
qry = \
"""
SELECT
    src_data,
    sum(IF(has(monthly.borr_asst_plan, 'F'), 1, 0)) AS forbearance,
    sum(IF(has(monthly.borr_asst_plan, 'R'), 1, 0)) AS payplan,
    count(*) AS nl
FROM
    unified.frannie
WHERE
    ln_defrl_amt > 0
GROUP BY src_data
ORDER BY src_data
"""

dfx1 = chu.run_query(qry, client, return_df=True)
dfx1.head(n=1000)

,src_data,forbearance,payplan,nl
0,fannie,209495,2293,210800
1,freddie:non-standard,20354,9466,88241
2,freddie:standard,186219,16842,268046
